In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import dataset
df = pd.read_csv('international_matches.csv')
df.tournament.value_counts()

# filter to just FIFA World Cup or FIFA World Cup qualification
wc = df[(df.tournament == 'FIFA World Cup') | (df.tournament == 'FIFA World Cup qualification')]

In [3]:
# if at least 1 nan in the columns: home_team_goalkeeper_score, away_team_goalkeeper_score, home_team_mean_defense_score, home_team_mean_offense_score, home_team_mean_midfield_score, away_team_mean_defense_score, away_team_mean_offense_score, away_team_mean_midfield_score, then drop the row
wc_cleaned = wc.dropna(subset=['home_team_goalkeeper_score', 'away_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score', 'home_team_mean_midfield_score', 'away_team_mean_defense_score', 'away_team_mean_offense_score', 'away_team_mean_midfield_score'])

# check wc_cleaned
wc_cleaned

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8266,2004-09-04,Austria,England,Europe,Europe,90,7,0,0,2,...,No,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,0,0,3,...,No,Win,77.0,74.0,80.5,78.7,79.0,71.8,75.7,70.2
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,0,0,1,...,No,Lose,78.0,78.0,68.8,77.0,69.2,70.5,79.7,78.5
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,0,0,2,...,No,Win,97.0,79.0,91.8,92.3,87.5,79.2,81.3,79.0
8283,2004-09-04,Morocco,Tunisia,Africa,Africa,33,36,0,0,1,...,No,Draw,68.0,51.0,75.2,73.7,71.0,67.5,71.0,72.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23632,2022-03-30,Panama,Canada,North America,North America,63,33,1375,1497,1,...,No,Win,75.0,76.0,69.8,70.7,67.5,69.2,73.0,78.0
23654,2022-06-01,Scotland,Ukraine,Europe,Europe,39,27,1472,1535,1,...,No,Lose,77.0,75.0,79.2,74.0,77.5,74.8,78.7,80.0
23723,2022-06-05,Wales,Ukraine,Europe,Europe,18,27,1588,1535,1,...,No,Win,74.0,75.0,75.0,73.0,78.5,74.8,78.7,80.0
23876,2022-06-13,Australia,Peru,Oceania,South America,42,22,1462,1562,0,...,Yes,Win,77.0,74.0,72.0,72.3,73.5,74.5,73.0,76.8


In [4]:
# import fifa_team_ranking.csv
team_rankings = pd.read_csv('fifa_team_ranking.csv')

# extract columns: rank, country_full, country_abrv, total_points, preivous_points, and rank_date from team_rankings
team_rankings = team_rankings[['rank', 'country_full', 'country_abrv', 'total_points', 'previous_points', 'rank_date']]
team_rankings

,rank,country_full,country_abrv,total_points,previous_points,rank_date
0,1,Germany,GER,0.0,57,1993-08-08
1,2,Italy,ITA,0.0,57,1993-08-08
2,3,Switzerland,SUI,0.0,50,1993-08-08
3,4,Sweden,SWE,0.0,55,1993-08-08
4,5,Argentina,ARG,0.0,51,1993-08-08
...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,2018-06-07
57789,206,Bahamas,BAH,0.0,0,2018-06-07
57790,206,Eritrea,ERI,0.0,0,2018-06-07
57791,206,Somalia,SOM,0.0,0,2018-06-07


## Experiment Imputing for Spain Matches

In [5]:
# check where spain played either as the home or away team
spain_games = wc_cleaned[(wc_cleaned.home_team == 'Spain') | (wc_cleaned.away_team == 'Spain')]

# extract first date of spain_games
match_date = spain_games.date.min()
match_date

'2004-09-08'

In [6]:
# check fifa rankings of spain
spain_rankings = team_rankings[team_rankings.country_full == 'Spain']

# find rank_date > match_date
spain_rankings_afterMatch = spain_rankings[spain_rankings.rank_date > match_date]
spain_rankings_afterMatch

# take the previous_points from the first row of spain_rankings_afterMatch
spain_rankings_afterMatch.previous_points.iloc[0]

788

## Expanding to all Matches

In [7]:
# replace FYR Macedonia in home_team and away_team with North Macedonia in team_rankings
team_rankings.replace('FYR Macedonia', 'North Macedonia', inplace=True)

In [8]:
# for a row in wc_cleaned, find the home and away team
for index, row in wc_cleaned.iterrows():

    # if the home_team_total_points is zero AND away_team_total_points is non-zero, then fill in the home_team_total_points from team_rankings
    if row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points != 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[0]

    # if the home_team_total_points is non-zero AND away_team_total_points is zero, then fill in the away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points != 0 and row.away_team_total_fifa_points == 0:
        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[0]

    # if both home_team_total_points and away_team_total_points are zero, then fill in both home_team_total_points and away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points == 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[0]

        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[0]

    # if both home_team_total_points and away_team_total_points are non-zero, then do nothing
    else:
        pass

# check wc_cleaned    
wc_cleaned

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8266,2004-09-04,Austria,England,Europe,Europe,90,7,488,732,2,...,No,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,662,523,3,...,No,Win,77.0,74.0,80.5,78.7,79.0,71.8,75.7,70.2
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,514,595,1,...,No,Lose,78.0,78.0,68.8,77.0,69.2,70.5,79.7,78.5
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,722,610,2,...,No,Win,97.0,79.0,91.8,92.3,87.5,79.2,81.3,79.0
8283,2004-09-04,Morocco,Tunisia,Africa,Africa,33,36,642,631,1,...,No,Draw,68.0,51.0,75.2,73.7,71.0,67.5,71.0,72.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23632,2022-03-30,Panama,Canada,North America,North America,63,33,1375,1497,1,...,No,Win,75.0,76.0,69.8,70.7,67.5,69.2,73.0,78.0
23654,2022-06-01,Scotland,Ukraine,Europe,Europe,39,27,1472,1535,1,...,No,Lose,77.0,75.0,79.2,74.0,77.5,74.8,78.7,80.0
23723,2022-06-05,Wales,Ukraine,Europe,Europe,18,27,1588,1535,1,...,No,Win,74.0,75.0,75.0,73.0,78.5,74.8,78.7,80.0
23876,2022-06-13,Australia,Peru,Oceania,South America,42,22,1462,1562,0,...,Yes,Win,77.0,74.0,72.0,72.3,73.5,74.5,73.0,76.8


In [9]:
# check if there are any zero values in home_team_total_fifa_points and away_team_total_fifa_points
wc_cleaned[(wc_cleaned.home_team_total_fifa_points == 0) | (wc_cleaned.away_team_total_fifa_points == 0)]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
9660,2006-06-11,Serbia,Netherlands,Europe,Europe,47,3,0,768,0,...,No,Lose,75.0,86.0,78.5,82.3,81.5,84.0,91.0,85.2
9673,2006-06-16,Argentina,Serbia,South America,Europe,9,47,746,0,6,...,No,Win,87.0,75.0,86.5,86.0,87.0,78.5,82.3,81.5
9690,2006-06-21,Côte d'Ivoire,Serbia,Africa,Europe,32,47,669,0,3,...,No,Win,69.0,75.0,81.2,81.7,75.8,78.5,82.3,81.5


In [10]:
# for a row in wc_cleaned, find the home and away team and impute with the 2nd most immediate previous points after the match date
for index, row in wc_cleaned.iterrows():

    # if the home_team_total_points is zero AND away_team_total_points is non-zero, then fill in the home_team_total_points from team_rankings
    if row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points != 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[1]

    # if the home_team_total_points is non-zero AND away_team_total_points is zero, then fill in the away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points != 0 and row.away_team_total_fifa_points == 0:
        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[1]

    # if both home_team_total_points and away_team_total_points are zero, then fill in both home_team_total_points and away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points == 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[1]

        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[1]

    # if both home_team_total_points and away_team_total_points are non-zero, then do nothing
    else:
        pass

In [11]:
# find match where home_team = Germany, away_team = Costa Rica
wc_cleaned[(wc_cleaned.home_team == 'Germany') & (wc_cleaned.away_team == 'Costa Rica')]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
9653,2006-06-09,Germany,Costa Rica,Europe,North America,19,26,696,683,4,...,No,Win,93.0,72.0,86.2,85.0,86.0,69.5,70.0,68.8
